In [15]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

%matplotlib inline
import geopandas as gpd
import altair as alt
import pandas as pd
import cenpy as cen

In [3]:
# Read the shape file, giving the name of the directory
NPS = gpd.read_file("NPS_-_National_Register_of_Historic_Places_Locations-shp/NPS_-_National_Register_of_Historic_Places_Locations.shp")
NPS.head()

,OBJECTID,NRIS_Refnu,PROPERTY_I,RESNAME,ResType,Address,City,County,State,Vicinity,...,SOURCE,SRC_DATE,SRC_SCALE,SRC_ACCU,SRC_COORD,ORIGINATOR,CONSTRANT,CR_ID,GEOM_ID,geometry
0,1,82004903,82004903,Walker-Broderick House,Building,541 Pine St.,Ketchikan,Ketchikan Gateway,ALASKA,None,...,National Register of Historic Places -- Nation...,1982-08-31,1:24000,+/- 12 meters,Coordinates are typically provided in UTM form...,National Register of Historic Places -- Nation...,Extant status and datum information for resour...,{5B428330-08FF-46F7-867D-35C3A84FDF0F},{D6FD98D4-BE9E-4C31-9F3D-095AFFCFEEB1},POINT (-131.64676 55.34455)
1,2,87000716,87000716,First Lutheran Church,Building,1200 Tongass Ave.,Ketchikan,Ketchikan Gateway,ALASKA,None,...,National Register of Historic Places -- Nation...,1987-05-18,1:24000,+/- 12 meters,Coordinates are typically provided in UTM form...,National Register of Historic Places -- Nation...,Extant status and datum information for resour...,{4B2E47FE-717C-4956-B460-2CA535A72A41},{CE99B08D-903E-4DEC-8247-BA95F9A6C1C9},POINT (-131.65777 55.34521)
2,3,87000645,87000645,Ketchikan Ranger House,Building,309 Gorge St.,Ketchikan,Ketchikan Gateway,ALASKA,None,...,National Register of Historic Places -- Nation...,1987-07-16,1:24000,+/- 12 meters,Coordinates are typically provided in UTM form...,National Register of Historic Places -- Nation...,Extant status and datum information for resour...,{23DF20B8-32F5-4F6C-861B-3188B30FD29F},{4413C896-7887-4201-95E1-7305CA794395},POINT (-131.65928 55.34652)
3,4,89001415,89001415,Gilmore Building,Building,326 Front St.,Ketchikan,Ketchikan Gateway,ALASKA,None,...,National Register of Historic Places -- Nation...,1989-09-27,1:24000,+/- 12 meters,Coordinates are typically provided in UTM form...,National Register of Historic Places -- Nation...,Extant status and datum information for resour...,{5BFBE8FE-BBDE-47AA-BC08-AE21BB63E16C},{59DA3A90-4DCF-4967-A6A9-6F56DD622D41},POINT (-131.64784 55.34230)
4,5,85001161,85001161,Ziegler House,Building,623 Grant St.,Ketchikan,Ketchikan Gateway,ALASKA,None,...,National Register of Historic Places -- Nation...,1985-05-30,1:24000,+/- 12 meters,Coordinates are typically provided in UTM form...,National Register of Historic Places -- Nation...,Extant status and datum information for resour...,{8D61329A-E9E4-4128-85C8-025585EEC7B8},{54B59265-DAFA-4BB3-A55A-1BB21F24B027},POINT (-131.64534 55.34447)


In [40]:
# Lets review all existing columns:
NPS.columns

Index(['OBJECTID', 'NRIS_Refnu', 'PROPERTY_I', 'RESNAME', 'ResType', 'Address',
       'City', 'County', 'State', 'Vicinity', 'MultiName', 'NumCBldg',
       'NumCObj', 'NumCSite', 'NumCStru', 'Status', 'Is_NHL', 'Status_Dat',
       'Listed_Dat', 'NARA_URL', 'BND_TYPE', 'BND_OTHER', 'IS_EXTANT',
       'EXTANT_OTH', 'CREATEDATE', 'EDIT_DATE', 'MAP_METHOD', 'MAP_MTH_OT',
       'SOURCE', 'SRC_DATE', 'SRC_SCALE', 'SRC_ACCU', 'SRC_COORD',
       'ORIGINATOR', 'CONSTRANT', 'CR_ID', 'GEOM_ID', 'geometry'],
      dtype='object')

In [41]:
# query the unique values of 'status' column:

NPS.Status.unique()

array(['Listed', 'Removed', None], dtype=object)

In [42]:
# I want to know how many from each one:

NPS['Status'].value_counts()

#there are 181 removed that we wish to drop from the dataset before starting. 
#We also wish to clean rows with no geometry (if there are)

Listed     56716
Removed      181
Name: Status, dtype: int64

In [43]:
len(NPS)

67443

In [44]:
NPS = NPS[NPS['Status']!='Removed']

len(NPS)

67262

In [51]:
# remove NaNs
NPS = NPS[NPS['geometry'].isna() != True]

len(NPS)

67244

In [54]:
grped = NPS.groupby('State').count().reset_index()
grped

,State,OBJECTID,NRIS_Refnu,PROPERTY_I,RESNAME,ResType,Address,City,County,Vicinity,...,SOURCE,SRC_DATE,SRC_SCALE,SRC_ACCU,SRC_COORD,ORIGINATOR,CONSTRANT,CR_ID,GEOM_ID,geometry
0,ALABAMA,843,843,843,843,843,843,843,843,110,...,843,843,843,843,843,843,843,843,843,843
1,ALASKA,286,286,286,286,286,286,286,286,77,...,286,286,286,286,286,286,286,286,286,286
2,AMERICAN SAMOA,18,18,18,18,18,18,18,18,8,...,18,18,18,18,18,18,18,18,18,18
3,ARIZONA,1023,1023,1023,1023,1023,1023,1023,1023,165,...,1023,1023,1023,1023,1023,1023,1023,1023,1023,1023
4,ARKANSAS,2264,2264,2264,2264,2264,2264,2264,2264,436,...,2264,2264,2264,2264,2264,2264,2264,2264,2264,2264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,VIRGINIA,1492,1492,1492,1492,1492,1492,1492,1492,428,...,1492,1492,1492,1492,1492,1492,1492,1492,1492,1492
58,WASHINGTON,1244,1244,1244,1244,1244,1244,1244,1244,237,...,1244,1244,1244,1244,1244,1244,1244,1244,1244,1244
59,WEST VIRGINIA,766,766,766,766,766,766,766,766,244,...,766,766,766,766,766,766,766,766,766,766
60,WISCONSIN,1788,1788,1788,1788,1788,1788,1788,1788,168,...,1788,1788,1788,1788,1788,1788,1788,1788,1788,1788


In [59]:
alt.Chart(grped).mark_bar().encode(
    x='State:N',
    y=alt.Y('OBJECTID', title='Count'),
)

alt.Chart(...)

In [12]:
# Add census data
acs = cenpy.remote.APIConnection("ACSDT5Y2018")

In [22]:
acs.varslike("POPULATION", by='concept').sort_index() # searches along concept column

,label,concept,predicateType,group,limit,predicateOnly,attributes,required
B00001_001E,Estimate!!Total,UNWEIGHTED SAMPLE COUNT OF THE POPULATION,NaN,B00001,0,NaN,B00001_001EA,NaN
B01003_001E,Estimate!!Total,TOTAL POPULATION,int,B01003,0,NaN,"B01003_001EA,B01003_001M,B01003_001MA",NaN
B05006PR_001E,Estimate!!Total,PLACE OF BIRTH FOR THE FOREIGN-BORN POPULATION...,int,B05006PR,0,NaN,"B05006PR_001EA,B05006PR_001M,B05006PR_001MA",NaN
B05006PR_002E,Estimate!!Total!!Europe,PLACE OF BIRTH FOR THE FOREIGN-BORN POPULATION...,int,B05006PR,0,NaN,"B05006PR_002EA,B05006PR_002M,B05006PR_002MA",NaN
B05006PR_003E,Estimate!!Total!!Europe!!Northern Europe,PLACE OF BIRTH FOR THE FOREIGN-BORN POPULATION...,int,B05006PR,0,NaN,"B05006PR_003EA,B05006PR_003M,B05006PR_003MA",NaN
...,...,...,...,...,...,...,...,...
C24070_081E,Estimate!!Total!!Self-employed in own not inco...,INDUSTRY BY CLASS OF WORKER FOR THE CIVILIAN E...,int,C24070,0,NaN,"C24070_081M,C24070_081MA,C24070_081EA",NaN
C24070_082E,Estimate!!Total!!Self-employed in own not inco...,INDUSTRY BY CLASS OF WORKER FOR THE CIVILIAN E...,int,C24070,0,NaN,"C24070_082M,C24070_082MA,C24070_082EA",NaN
C24070_083E,Estimate!!Total!!Self-employed in own not inco...,INDUSTRY BY CLASS OF WORKER FOR THE CIVILIAN E...,int,C24070,0,NaN,"C24070_083M,C24070_083MA,C24070_083EA",NaN
C24070_084E,Estimate!!Total!!Self-employed in own not inco...,INDUSTRY BY CLASS OF WORKER FOR THE CIVILIAN E...,int,C24070,0,NaN,"C24070_084M,C24070_084MA,C24070_084EA",NaN


In [23]:
variables = [
    "NAME",
    "B01003_001E" # Total population
]

In [31]:
state_data = acs.query(
    cols=variables,
    geo_unit="state:*",
)

In [32]:
for variable in variables:
    
    # Convert all variables EXCEPT for NAME
    if variable != "NAME":
        state_data[variable] = state_data[variable].astype(float)

In [39]:
state_data.head()

,NAME,B01003_001E,state
0,Minnesota,5527358.0,27
1,Mississippi,2988762.0,28
2,Missouri,6090062.0,29
3,Montana,1041732.0,30
4,Nebraska,1904760.0,31


In [35]:
ny_state_code = "36"

In [36]:
ny_data = acs.query(
    cols=variables,
    geo_unit="county:*",
    geo_filter={"state": ny_state_code},
)

In [37]:
for variable in variables:
    
    # Convert all variables EXCEPT for NAME
    if variable != "NAME":
        ny_data[variable] = ny_data[variable].astype(float)

In [38]:
ny_data.head()

,NAME,B01003_001E,state,county
0,"Schoharie County, New York",31364.0,36,095
1,"Onondaga County, New York",464242.0,36,067
2,"Clinton County, New York",80794.0,36,019
3,"Seneca County, New York",34612.0,36,099
4,"Putnam County, New York",99070.0,36,079
